# T81-558: Applications of Deep Neural Networks
**Module 12: Deep Learning and Security**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 12 Video Material

* Part 12.1: Introduction to the OpenAI Gym [[Video]](https://www.youtube.com/watch?v=_KbUxgyisjM&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_01_ai_gym.ipynb)
* Part 12.2: Introduction to Q-Learning [[Video]](https://www.youtube.com/watch?v=uwcXWe_Fra0&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_02_qlearningreinforcement.ipynb)
* **Part 12.3: Keras Q-Learning in the OpenAI Gym** [[Video]](https://www.youtube.com/watch?v=Ya1gYt63o3M&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_03_keras_reinforce.ipynb)
* Part 12.4: Atari Games with Keras Neural Networks [[Video]](https://www.youtube.com/watch?v=t2yIu6cRa38&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_04_atari.ipynb)
* Part 12.5: How Alpha Zero used Reinforcement Learning to Master Chess [[Video]](https://www.youtube.com/watch?v=ikDgyD7nVI8&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_12_05_alpha_zero.ipynb)


# Part 12.3: Keras Q-Learning in the OpenAI Gym

![Deep Q-Learning](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/deepqlearning.png "Reinforcement Learning")

* **CEMAgent**
    * **model** - The neural network that will be trained.
    * **nb_actions** - The number of actions the agent can take (e.g. up, down, left, right, fire)
    * **memory** - The EpisodeParameterMemory object to use.  This object observes and save all of the state transitions so that you can train your network on them later on (instead of having to make observations from the environment all the time).
    * **batch_size** - The batch size for neural network training, same concept as deep learning batch sizes.
    * **nb_steps_warmup** - Number of training steps to pass before any learning occurs.
    * **train_interval** - Logging interval, defines how often to log.
    * **elite_frac**
* **CEMAgent.fit**
    * **env** - The OpenAI gym environment being used.
    * **nb_steps** - Number of training steps to be performed.
    * **visualize** - If `True`, the environment is visualized during training. However,
                this is likely going to slow down training significantly and is thus intended to be
                a debugging instrument.
    * **verbose** - 0 for no logging, 1 for interval logging (compare `log_interval`), 2 for episode logging




In [1]:
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents.cem import CEMAgent
from rl.memory import EpisodeParameterMemory

ENV_NAME = 'CartPole-v0'


# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)

nb_actions = env.action_space.n
obs_dim = env.observation_space.shape[0]

# Option 1 : Simple model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))

# Option 2: deep network
# model = Sequential()
# model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(nb_actions))
# model.add(Activation('softmax'))


print(model.summary())


# Finally, we configure and compile our agent. You can use every built-in tensorflow.keras optimizer and
# even the metrics!
memory = EpisodeParameterMemory(limit=1000, window_length=1)

cem = CEMAgent(model=model, nb_actions=nb_actions, memory=memory,
               batch_size=50, nb_steps_warmup=2000, train_interval=50, elite_frac=0.05)
cem.compile()

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
cem.fit(env, nb_steps=100000, visualize=False, verbose=2)

# After training is done, we save the best weights.
cem.save_weights('cem_{}_params.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
cem.test(env, nb_episodes=5, visualize=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 2)                 10        
_________________________________________________________________
activation (Activation)      (None, 2)                 0         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
None
Training for 100000 steps ...
    28/100000: episode: 1, duration: 0.049s, episode steps:  28, steps per second: 567, episode reward: 28.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
    41/100000: episode: 2, duration: 0.008s, episode steps:  13, steps per second: 1537, episode reward: 13.000, mean reward:  1.000 [ 1.000,  1.000

   883/100000: episode: 45, duration: 0.018s, episode steps:  31, steps per second: 1678, episode reward: 31.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.516 [0.000, 1.000],  mean_best_reward: --
   902/100000: episode: 46, duration: 0.015s, episode steps:  19, steps per second: 1245, episode reward: 19.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.368 [0.000, 1.000],  mean_best_reward: --
   917/100000: episode: 47, duration: 0.011s, episode steps:  15, steps per second: 1343, episode reward: 15.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.333 [0.000, 1.000],  mean_best_reward: --
   945/100000: episode: 48, duration: 0.016s, episode steps:  28, steps per second: 1728, episode reward: 28.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.679 [0.000, 1.000],  mean_best_reward: --
   974/100000: episode: 49, duration: 0.018s, episode steps:  29, steps per second: 1607, episode reward: 29.000, mean reward:  1.000 [ 1.000,  1.000], mean action:

  1848/100000: episode: 97, duration: 0.028s, episode steps:  48, steps per second: 1735, episode reward: 48.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.458 [0.000, 1.000],  mean_best_reward: --
  1866/100000: episode: 98, duration: 0.012s, episode steps:  18, steps per second: 1549, episode reward: 18.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.333 [0.000, 1.000],  mean_best_reward: --
  1882/100000: episode: 99, duration: 0.011s, episode steps:  16, steps per second: 1498, episode reward: 16.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.312 [0.000, 1.000],  mean_best_reward: --
  1897/100000: episode: 100, duration: 0.011s, episode steps:  15, steps per second: 1414, episode reward: 15.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.133 [0.000, 1.000],  mean_best_reward: --
  1910/100000: episode: 101, duration: 0.009s, episode steps:  13, steps per second: 1490, episode reward: 13.000, mean reward:  1.000 [ 1.000,  1.000], mean actio

  2912/100000: episode: 147, duration: 0.020s, episode steps:  38, steps per second: 1899, episode reward: 38.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.421 [0.000, 1.000],  mean_best_reward: --
  2925/100000: episode: 148, duration: 0.009s, episode steps:  13, steps per second: 1482, episode reward: 13.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.308 [0.000, 1.000],  mean_best_reward: --
  3000/100000: episode: 149, duration: 0.036s, episode steps:  75, steps per second: 2068, episode reward: 75.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.467 [0.000, 1.000],  mean_best_reward: --
  3037/100000: episode: 150, duration: 0.019s, episode steps:  37, steps per second: 1985, episode reward: 37.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.351 [0.000, 1.000],  mean_best_reward: --
  3054/100000: episode: 151, duration: 0.010s, episode steps:  17, steps per second: 1648, episode reward: 17.000, mean reward:  1.000 [ 1.000,  1.000], mean ac

  4012/100000: episode: 204, duration: 0.008s, episode steps:  12, steps per second: 1490, episode reward: 12.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.583 [0.000, 1.000],  mean_best_reward: --
  4030/100000: episode: 205, duration: 0.011s, episode steps:  18, steps per second: 1664, episode reward: 18.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.556 [0.000, 1.000],  mean_best_reward: --
  4043/100000: episode: 206, duration: 0.007s, episode steps:  13, steps per second: 1839, episode reward: 13.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.308 [0.000, 1.000],  mean_best_reward: --
  4061/100000: episode: 207, duration: 0.011s, episode steps:  18, steps per second: 1695, episode reward: 18.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
  4111/100000: episode: 208, duration: 0.026s, episode steps:  50, steps per second: 1951, episode reward: 50.000, mean reward:  1.000 [ 1.000,  1.000], mean ac

...

 91872/100000: episode: 2695, duration: 0.031s, episode steps:  68, steps per second: 2164, episode reward: 68.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
 91968/100000: episode: 2696, duration: 0.045s, episode steps:  96, steps per second: 2119, episode reward: 96.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.510 [0.000, 1.000],  mean_best_reward: --
 92016/100000: episode: 2697, duration: 0.024s, episode steps:  48, steps per second: 2032, episode reward: 48.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.521 [0.000, 1.000],  mean_best_reward: --
 92039/100000: episode: 2698, duration: 0.012s, episode steps:  23, steps per second: 2000, episode reward: 23.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.435 [0.000, 1.000],  mean_best_reward: --
 92134/100000: episode: 2699, duration: 0.043s, episode steps:  95, steps per second: 2192, episode reward: 95.000, mean reward:  1.000 [ 1.000,  1.000], me

 93750/100000: episode: 2740, duration: 0.033s, episode steps:  66, steps per second: 2006, episode reward: 66.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
 93792/100000: episode: 2741, duration: 0.020s, episode steps:  42, steps per second: 2104, episode reward: 42.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
 93870/100000: episode: 2742, duration: 0.036s, episode steps:  78, steps per second: 2174, episode reward: 78.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.487 [0.000, 1.000],  mean_best_reward: --
 93913/100000: episode: 2743, duration: 0.020s, episode steps:  43, steps per second: 2178, episode reward: 43.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.488 [0.000, 1.000],  mean_best_reward: --
 93981/100000: episode: 2744, duration: 0.032s, episode steps:  68, steps per second: 2134, episode reward: 68.000, mean reward:  1.000 [ 1.000,  1.000], me

 95570/100000: episode: 2786, duration: 0.063s, episode steps: 136, steps per second: 2152, episode reward: 136.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
 95588/100000: episode: 2787, duration: 0.010s, episode steps:  18, steps per second: 1894, episode reward: 18.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.444 [0.000, 1.000],  mean_best_reward: --
 95608/100000: episode: 2788, duration: 0.010s, episode steps:  20, steps per second: 1969, episode reward: 20.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.550 [0.000, 1.000],  mean_best_reward: --
 95627/100000: episode: 2789, duration: 0.009s, episode steps:  19, steps per second: 2014, episode reward: 19.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.474 [0.000, 1.000],  mean_best_reward: --
 95651/100000: episode: 2790, duration: 0.012s, episode steps:  24, steps per second: 1993, episode reward: 24.000, mean reward:  1.000 [ 1.000,  1.000], m

 97339/100000: episode: 2833, duration: 0.037s, episode steps:  78, steps per second: 2092, episode reward: 78.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.500 [0.000, 1.000],  mean_best_reward: --
 97370/100000: episode: 2834, duration: 0.015s, episode steps:  31, steps per second: 2001, episode reward: 31.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.516 [0.000, 1.000],  mean_best_reward: --
 97411/100000: episode: 2835, duration: 0.019s, episode steps:  41, steps per second: 2118, episode reward: 41.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.463 [0.000, 1.000],  mean_best_reward: --
 97436/100000: episode: 2836, duration: 0.014s, episode steps:  25, steps per second: 1820, episode reward: 25.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.560 [0.000, 1.000],  mean_best_reward: --
 97464/100000: episode: 2837, duration: 0.013s, episode steps:  28, steps per second: 2075, episode reward: 28.000, mean reward:  1.000 [ 1.000,  1.000], me

 99490/100000: episode: 2881, duration: 0.016s, episode steps:  34, steps per second: 2141, episode reward: 34.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.529 [0.000, 1.000],  mean_best_reward: --
 99507/100000: episode: 2882, duration: 0.010s, episode steps:  17, steps per second: 1665, episode reward: 17.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.529 [0.000, 1.000],  mean_best_reward: --
 99604/100000: episode: 2883, duration: 0.045s, episode steps:  97, steps per second: 2159, episode reward: 97.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.495 [0.000, 1.000],  mean_best_reward: --
 99646/100000: episode: 2884, duration: 0.020s, episode steps:  42, steps per second: 2065, episode reward: 42.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 0.524 [0.000, 1.000],  mean_best_reward: --
 99685/100000: episode: 2885, duration: 0.018s, episode steps:  39, steps per second: 2143, episode reward: 39.000, mean reward:  1.000 [ 1.000,  1.000], me